Requirements:
```
pip install vaex-jupyter==0.5.1 vaex-server==0.3.1
```

In [ ]:
import os

import vaex
import numpy as np
import ipyvuetify as v

We will use the remote dataframe:

In [ ]:
powerful_machine = False
if powerful_machine:
    # If we are on a powerful machine, we can download and process the data locally
    profile_name = 'stsci' if getpass.getuser() == 'maartenbreddels' else 'default'
    df = vaex.open(f's3://astrosurveydata/gaia_ps1_nochunk.hdf5?profile_name={profile_name}')
else:
    # otherwise we connect to the STScI Gaia machine (a remote dataframe)
#     # it seems from mybinder we cannot reach this machine
#     path = 'ws://ec2-18-222-183-211.us-east-2.compute.amazonaws.com:9000/gaia_ps1_nochunk'
#     if os.path.exists('token_trusted.txt'):
#         with open('token_trusted.txt') as f:
#             token_trusted = f.read().strip()
#             path += f'?token_trusted={token_trusted}'
#     elif os.path.exists('token.txt'):
#         with open('token.txt') as f:
#             token = f.read().strip()
#             path += f'?token={token}'
    path = 'vaex+ws://dataframe.vaex.io/gaia_ps1_nochunk'
    df = vaex.open(path)
df

In [ ]:
df = df[:100_000_000]  # take a subset to make it faster

# Initialization
Set up some variables, and add a few columns

In [ ]:
limits_l = [360, 0]
limits_b = [-90, 90]

color = df.phot_bp_mean_mag - df.phot_g_mean_mag
df['distance'] = 1/df.parallax
df['M_g'] = df.phot_g_mean_mag - 5 * np.log10(df.distance) - 10
magnitude = df.M_g
limits_color = [-0.5, 1.5]
limits_magnitude = [22, -10]
df['magnitude'] = magnitude
df['color'] = color

aux = df.phot_g_mean_mag
limits_aux = [5, 23]
shape_aux = 100
limits_l = [360, 0]
limits_b = [-90, 90]

# make a first selection
# df.select(df.distance < 10)
df.select(df.color < 1)

# Heatmap 1
Our first heatmap is a CMD

In [ ]:
heatmap_cmd = df.widget.heatmap(df.color, df.magnitude,
                                limits=[limits_color, limits_magnitude],
                                selections=[None, 'default'])
heatmap_cmd

Where the x and y axis can be chosen by the user.

In [ ]:
x_widget = df.widget.column(heatmap_cmd.model.x, label='X axis')
y_widget = df.widget.column(heatmap_cmd.model.y, label='Y axis')

And we put it into a nice container.

In [ ]:
card1 = vaex.jupyter.widgets.ContainerCard(title='Heatmap 1',
              main=heatmap_cmd,
              controls=[x_widget, y_widget],
              show_controls=True,
              card_props={'style': 'min-width: 350px;'})
card1

In [ ]:
# It shouldn't be needed to wait for the first plot to finish
# However, it sometimes hangs the 2nd heatmap, needs to be fixed!
await vaex.jupyter.gather()

# Heatmap 2
Our second heatmap always shows the sky.

In [ ]:
heatmap_sky = df.widget.heatmap(df.l, df.b, limits=[limits_l, limits_b], selection=[None, 'default'])
heatmap_sky

Again, shown in a card.

In [ ]:
card2 = vaex.jupyter.widgets.ContainerCard(title='Heatmap 2',
              main=heatmap_sky,
              controls=[],
              show_controls=False,
              card_props={'style': 'min-width: 350px;'})
card2

# Custom selection widget
To allow for arbitrary selections, we create a widget to edit the selection.

In [ ]:
widget_selection_expression = df.widget.selection_expression()

In [ ]:
card3 = vaex.jupyter.widgets.ContainerCard(title='Custom selection',
              main=widget_selection_expression,
              controls=[],
              show_controls=False,
              card_props={'style': 'min-width: 300px;'}
)
card3

# Voila-vuetify
This part is only used for voila-vuetify, where the mount_id specify where the widgets are placed.
Run as:
```
$ voila --enable_nbextensions=True --template vuetify-default ~/src/maartenbreddels/gde-examples/gaia-voila-vaex-jupyter-simple.ipynb
```

In [ ]:
# all cards are 12 columns wide on small screen, 6 on medium, 4 or large
# see https://vuetifyjs.com/en/components/grids/#grid-system
v.Container(fluid=True, _metadata={'mount_id': 'content-main'}, children=[
    v.Row(children=[
        v.Col(lg=4, md=6, sm=12, children=[
            card1
        ]),
        v.Col(lg=4, md=6, sm=12, children=[
            card2
        ]),
        v.Col(lg=4, md=6, sm=12, children=[
            card3
        ]),
    ])
])

The column list goes into the navigation panel

In [ ]:
column_list = vaex.jupyter.widgets.ColumnList(df=df, _metadata={'mount_id': 'content-nav'}, )
column_list